In [1]:
api= ""
myid= ""

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import time
import json
import requests
from nutritionix import Nutritionix
nix = Nutritionix(app_id= myid, api_key=api)
pd.options.display.float_format = '{:,.1f}'.format

In [3]:
# Start with an empty list.
u_inputs = []
user = []
# Set u_input to something other than 'nothing'.
user1= input("Hi there! What is your name? ")
user.append(user1)
print(f"Hi {user1}! I am going to ask you a couple questions")
u_age= input("How old are you? ")
u_weight= int(input("What do you weigh in pounds? "))
u_input = input("What are you considering eating today?: ")
u_inputs.append(u_input)
# Start a loop that will run until the user enters 'nothing'.
while u_input != "no":
    # Ask the user for a new meal.
    u_input= input("Is there anything else you want?, or say no: ")

    # Add the new meal to our list.
    if u_input != 'no':
        u_inputs.append(u_input)

Hi there! What is your name? Alex
Hi Alex! I am going to ask you a couple questions
How old are you? 28
What do you weigh in pounds? 200
What are you considering eating today?: Chicken Sandwich
Is there anything else you want?, or say no: Apple
Is there anything else you want?, or say no: Orange
Is there anything else you want?, or say no: no


In [4]:
#u_inputs printed out
uu_inputs= '& '.join(u_inputs)

In [5]:
#Build Nutrition Value Data Frame table
nutri_pd= pd.DataFrame()
nutri_pd["Item Name"] = u_inputs
nutri_pd["Serving Size"] = ""
nutri_pd["Calories"] = ""
nutri_pd["Total Fat"] = ""
nutri_pd["Cholesterol"] = ""
nutri_pd["Sodium"] = ""
nutri_pd["Total Carbohydrate"] = ""
nutri_pd["Fiber"] = ""
nutri_pd["Sugar"] = ""
nutri_pd["Protein"] = ""

In [6]:
index= 0 #index for nutri pd
nutri=[]
for index, row in nutri_pd.iterrows():
    food= row['Item Name']
    search= nix.search(food).json()
    #results = nix.item(id= search['hits'][0]['_id']).json()
    
    try:
        results = nix.item(id= search['hits'][0]['_id']).json()
        #print(results)
        nutri_pd.loc[index,"Calories"]= results['nf_calories']
        nutri_pd.loc[index,"Serving Size"]= results['nf_serving_size_qty']
        cal_fat=results['nf_calories_from_fat']
        nutri_pd.loc[index,"Total Fat"]= results['nf_total_fat']
        sat_fat= results['nf_saturated_fat']
        trans_fat= results['nf_trans_fatty_acid']
        poly_fat= results['nf_polyunsaturated_fat']
        mono_fat= results['nf_monounsaturated_fat']
        nutri_pd.loc[index,"Cholesterol"]= results['nf_cholesterol']
        nutri_pd.loc[index,"Sodium"]= results['nf_sodium']
        nutri_pd.loc[index,"Total Carbohydrate"]= results['nf_total_carbohydrate']
        nutri_pd.loc[index,"Fiber"]= results['nf_dietary_fiber']
        nutri_pd.loc[index,"Sugar"]= results['nf_sugars']
        nutri_pd.loc[index,"Protein"]= results['nf_protein']
        vit_a= results['nf_vitamin_a_dv']
        vit_c= results['nf_vitamin_c_dv']
        calc= results['nf_calcium_dv']
        iron= results['nf_iron_dv']
        s_size=  results['nf_serving_size_qty']
    except (KeyError, IndexError):
        print("Meal not found. Skipping...")

In [7]:
#nutri_pd

In [8]:
#Get daily recommended caloric intake
daily_cal= u_weight*12
daily_pro= u_weight*0.36
daily45_carb= daily_cal*0.45*0.25
daily65_carb= daily_cal*0.65*0.25
daily20_fat= daily_cal*0.20*(1/9)
daily35_fat= daily_cal*0.35*(1/9)
daily_na= "1.5-2.3"
rec_pd= pd.DataFrame()
rec_pd["Name"]= user
rec_pd["Age"]= u_age
rec_pd["Weight"]= u_weight
rec_pd["Daily Calories(cal)"]= daily_cal
rec_pd["Daily Protein(g)"]= daily_pro
rec_pd["Min Daily Carbs(g)"]= daily45_carb
rec_pd["Max Daily Carbs(g)"]= daily65_carb
rec_pd["Min Daily Fat(g)"]= daily20_fat
rec_pd["Max Daily Fat(g)"]= daily35_fat
rec_pd["Daily Sodium(g)"]= daily_na
#rec_pd

In [9]:
#Make dataframe for exercises
exercise_pd= pd.DataFrame()
exercise_pd["Item Name"]= nutri_pd["Item Name"]
exercise_pd["Calories"]= nutri_pd["Calories"]
exercise_pd["Walking(min)"]= ""
exercise_pd["Running(min)"]= ""
exercise_pd["Cycling(min)"]= ""
#exercise_pd

In [10]:
#Use API to get recommended exercises
headers = {'x-app-key': api,'x-app-id': myid, 'x-remote-user-id':"0"}
for index, row in exercise_pd.iterrows():
    #get calories for each meal
    meal_cal= row["Calories"]
    #make a list of types of exercise
    type_ex= ["Walking","Running","Cycling"]
    for ex in type_ex:
        payload={"query":f"{ex} {meal_cal} calories"}
        ex_req = requests.post("https://trackapi.nutritionix.com/v2/natural/exercise", headers= headers, data=payload)
        ex_req_json= ex_req.json()
        exercise_pd.loc[index,f"{ex}(min)"]= ex_req_json["exercises"][0]["duration_min"]
        #print(payload)
        #print(ex_req_json)
    

In [11]:
#ex_req_json["exercises"][0]["duration_min"]
#exercise_pd

In [12]:
#Total calories of what user will be eating
total_cal= nutri_pd["Calories"].sum(axis=0)
#total_cal

In [13]:
#Remaining calories
rem_cal= rec_pd["Daily Calories(cal)"][0]- total_cal
#rem_cal
#exercise_pd

In [14]:
#Total exercise based on calories to be eaten
total_ex_walk= exercise_pd["Walking(min)"].sum(axis=0)
total_ex_run= exercise_pd["Running(min)"].sum(axis=0)
total_ex_cyc= exercise_pd["Cycling(min)"].sum(axis=0)

In [15]:
print("--------------------------------")
print(f"{user1}'s Snapshot!!")
print("--------------------------------")
print(f"Hi {user1}, let's consider your user profile")
print("")
print(f"Your recommended total daily calorie intake is {rec_pd['Daily Calories(cal)'][0]} calories")
print("")
print(f"If you end up eating {uu_inputs}; you will have to keep your calorie intake under {rem_cal} calories")
print("")
print(f"If you end up eating {uu_inputs}; you would have to run for {total_ex_run} mins OR walk for {total_ex_walk} mins OR cycle for {total_ex_cyc} min")
print("")
print("Please refer to the tables below for a more detailed overview of our recommendations.")
print("")
print("--------------------------------")
print(f"General recommendations")
print("--------------------------------")
print("Consider reducing your total daily calories by 500 in order to lose a pound a week.")
print("You would need to reduce your total daily calories by 1000 in order to lose two pounds a week but this is highly unadvisable.")

--------------------------------
Alex's Snapshot!!
--------------------------------
Hi Alex, let's consider your user profile

Your recommended total daily calorie intake is 2400 calories

If you end up eating Chicken Sandwich& Apple& Orange; you will have to keep your calorie intake under 1746.4 calories

If you end up eating Chicken Sandwich& Apple& Orange; you would have to run for 57.169999999999995 mins OR walk for 160.07 mins OR cycle for 56.02 min

Please refer to the tables below for a more detailed overview of our recommendations.

--------------------------------
General recommendations
--------------------------------
Consider reducing your total daily calories by 500 in order to lose a pound a week.
You would need to reduce your total daily calories by 1000 in order to lose two pounds a week but this is highly unadvisable.


In [16]:
print(f"Hi {user1}, let's consider your user profile:")
rec_pd.set_index("Name")

Hi Alex, let's consider your user profile:


,Age,Weight,Daily Calories(cal),Daily Protein(g),Min Daily Carbs(g),Max Daily Carbs(g),Min Daily Fat(g),Max Daily Fat(g),Daily Sodium(g)
Name,,,,,,,,,
Alex,28,200,2400,72.0,270.0,390.0,53.3,93.3,1.5-2.3


In [17]:
print(f" So {user1}, lets run through all the things you want to eat:")
nutri_pd.set_index("Item Name")

 So Alex, lets run through all the things you want to eat:


,Serving Size,Calories,Total Fat,Cholesterol,Sodium,Total Carbohydrate,Fiber,Sugar,Protein
Item Name,,,,,,,,,
Chicken Sandwich,1,520,16,105,1350,57,2,10,35
Apple,1,65,0.2,0,1.2,17.3,3,13.0,0.3
Orange,1,68.6,0.2,0,1.4,17.6,3.1,11.9,1.3


In [19]:
print(f"To lose all those calories, consider exercising:")
exercise_pd.set_index("Item Name")

To lose all those calories, consider exercising:


,Calories,Walking(min),Running(min),Cycling(min)
Item Name,,,,
Chicken Sandwich,520,127.3,45.5,44.6
Apple,65,15.9,5.7,5.6
Orange,68.6,16.8,6,5.9
